In [1]:
import pandas as pd
import numpy as np
from itertools import combinations
from collections import Counter

There is a bit of indiscepancy on creating the universe of patent class pairs, so until it is cleared up, I'll do the calculations both ways and see what's up

# Way indicated on instructions

1. retrieve all design patents
2. retrieve all patent classes for each design patent

In [2]:
# final = pd.read_csv('data/final_no_drops.csv')
# final.head

3) List all pairwise combinations of patent classes for each design patent

In [3]:
data = [('D111111','F',2000),('D111111','E',2000),('D111111','A',2000),('D111111','B',2000),('D222222','B',2001),('D222222', 'A',2001),('D222222','H',2001),
       ('D333333','B',2001),('D333333', 'A',2001),('D333333','H',2001),('D444444','Z',2000),('D444444', 'A',2000),('D444444','B',2000)]
cols = ['id','class1','year']


In [4]:
df = pd.DataFrame(data, columns=cols)
# df

In [5]:
def combine(batch):
    """Combine all products within one batch into pairs"""
    return pd.Series(list(combinations(set(batch), 2)))

In [6]:
df = df.groupby(['id','year'])['class1'].apply(combine).reset_index(level=0).reset_index(level=0).reset_index(drop=True)
df

,year,id,class1
0,2000,D111111,"(F, A)"
1,2000,D111111,"(F, B)"
2,2000,D111111,"(F, E)"
3,2000,D111111,"(A, B)"
4,2000,D111111,"(A, E)"
5,2000,D111111,"(B, E)"
6,2001,D222222,"(A, H)"
7,2001,D222222,"(A, B)"
8,2001,D222222,"(H, B)"
9,2001,D333333,"(A, H)"


In [7]:
df[['c1','c2']] = df['class1'].apply(pd.Series).apply(sorted, axis=1)
df

,year,id,class1,c1,c2
0,2000,D111111,"(F, A)",A,F
1,2000,D111111,"(F, B)",B,F
2,2000,D111111,"(F, E)",E,F
3,2000,D111111,"(A, B)",A,B
4,2000,D111111,"(A, E)",A,E
5,2000,D111111,"(B, E)",B,E
6,2001,D222222,"(A, H)",A,H
7,2001,D222222,"(A, B)",A,B
8,2001,D222222,"(H, B)",B,H
9,2001,D333333,"(A, H)",A,H


In [8]:
df.drop('class1', inplace=True, axis=1)
# df

,year,id,c1,c2
0,2000,D111111,A,F
1,2000,D111111,B,F
2,2000,D111111,E,F
3,2000,D111111,A,B
4,2000,D111111,A,E
5,2000,D111111,B,E
6,2001,D222222,A,H
7,2001,D222222,A,B
8,2001,D222222,B,H
9,2001,D333333,A,H


4) record the two paired patent classes for each patent class pair

I'm not sure what this means, but I'm assuming its done, since it sounds like "just say which classes are listed in the class pair"

5) record the application year (using the priority year here) of each patent.

This is done in final

6) pull together all patent class pairs applied in the same year to construct a universe of class patent pairs for each year t as Ut. Calculate the commonness of each patent class pair in year

In [9]:
# Nt = Number of all journal pairs that year
Nt_count = df.groupby(['year'])['c1'].count()
df['Nt'] = df.apply(lambda x: Nt_count[x['year']], axis=1)
# df

,year,id,c1,c2,Nt
0,2000,D111111,A,F,9
1,2000,D111111,B,F,9
2,2000,D111111,E,F,9
3,2000,D111111,A,B,9
4,2000,D111111,A,E,9
5,2000,D111111,B,E,9
6,2001,D222222,A,H,6
7,2001,D222222,A,B,6
8,2001,D222222,B,H,6
9,2001,D333333,A,H,6


In [10]:
#Nijt = number of i-j pairs in Ut
Nijt_count = df.groupby(['year','c1','c2'])['id'].count()
df['Nijt'] = df.apply(lambda x: Nijt_count.loc[x['year'],x['c1'],x['c2']],axis=1)
# df

,year,id,c1,c2,Nt,Nijt
0,2000,D111111,A,F,9,1
1,2000,D111111,B,F,9,1
2,2000,D111111,E,F,9,1
3,2000,D111111,A,B,9,2
4,2000,D111111,A,E,9,1
5,2000,D111111,B,E,9,1
6,2001,D222222,A,H,6,2
7,2001,D222222,A,B,6,2
8,2001,D222222,B,H,6,2
9,2001,D333333,A,H,6,2


In [11]:
count_class = pd.DataFrame(df.groupby('year').apply(lambda x: pd.Series(Counter(x['c1'].tolist() + x['c2'].tolist()))))
df['Nit'] = df.apply(lambda x: count_class.loc[x['year'],x['c1']], axis=1)
df['Nij'] = df.apply(lambda x: count_class.loc[x['year'],x['c2']], axis=1)
# df

,year,id,c1,c2,Nt,Nijt,Nit,Nij
0,2000,D111111,A,F,9,1,5,3
1,2000,D111111,B,F,9,1,5,3
2,2000,D111111,E,F,9,1,3,3
3,2000,D111111,A,B,9,2,5,5
4,2000,D111111,A,E,9,1,5,3
5,2000,D111111,B,E,9,1,5,3
6,2001,D222222,A,H,6,2,4,4
7,2001,D222222,A,B,6,2,4,4
8,2001,D222222,B,H,6,2,4,4
9,2001,D333333,A,H,6,2,4,4


In [12]:
def commonness(row):
    return (row['Nijt'] * row['Nt'])/(row['Nit'] * row['Nij'])

In [13]:
df['commonness'] = df.apply(commonness, axis=1)
df

,year,id,c1,c2,Nt,Nijt,Nit,Nij,commonness
0,2000,D111111,A,F,9,1,5,3,0.60
1,2000,D111111,B,F,9,1,5,3,0.60
2,2000,D111111,E,F,9,1,3,3,1.00
3,2000,D111111,A,B,9,2,5,5,0.72
4,2000,D111111,A,E,9,1,5,3,0.60
5,2000,D111111,B,E,9,1,5,3,0.60
6,2001,D222222,A,H,6,2,4,4,0.75
7,2001,D222222,A,B,6,2,4,4,0.75
8,2001,D222222,B,H,6,2,4,4,0.75
9,2001,D333333,A,H,6,2,4,4,0.75
